***Import các thư viện cần thiết***

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import random

***Setting biến***

In [2]:
# Setting model name
tflite_model_name = "disease_prediction_model" # .tflite suffix
c_model_name = 'disease_prediction_model' # .h suffix

***Đọc data từ file***

In [3]:
# Đọc dữ liệu từ các file đã tạo
train_df = pd.read_csv("heart_spo2_train.csv")
val_df = pd.read_csv("heart_spo2_validation.csv")
test_df = pd.read_csv("heart_spo2_test.csv")

***Chuẩn hóa dữ liệu***

In [5]:
# Gộp tất cả dữ liệu để chuẩn hóa
full_df = pd.concat([train_df, val_df, test_df])
X_full = full_df[["Heart Rate", "Oxygen Saturation"]].values
y_full = full_df["Label"].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_full = scaler.fit_transform(X_full)

***Xác định kích thước từng tập và scaler***

In [6]:
# Lấy lại dữ liệu train, validation, test từ dữ liệu đã chuẩn hóa
train_size = len(train_df)
val_size = len(val_df)
test_size = len(test_df)

X_train, X_val, X_test = X_full[:train_size], X_full[train_size:train_size+val_size], X_full[train_size+val_size:]
y_train, y_val, y_test = y_full[:train_size], y_full[train_size:train_size+val_size], y_full[train_size+val_size:]

# Kiểm tra kích thước tập dữ liệu
print(f"Kích thước tập train: {X_train.shape}")
print(f"Kích thước tập validation: {X_val.shape}")
print(f"Kích thước tập test: {X_test.shape}")

# Kiểm tra scaler
print("Scaler mean:", scaler.mean_)
print("Scaler scale:", scaler.scale_)


Kích thước tập train: (12000, 2)
Kích thước tập validation: (4000, 2)
Kích thước tập test: (4000, 2)
Scaler mean: [84.50905    93.29880835]
Scaler scale: [31.49859867  5.77284963]


***Xây dựng mô hình cho phân loại đa lớp (multi-class)***

In [7]:
model = Sequential([
    Dense(16, activation='relu', input_shape=(2,)),   
    Dense(16, activation='relu'),
    Dense(4, activation='softmax')
])

***Biên dịch mô hình với optimizer tùy chỉnh***

In [8]:
optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',  # Loss cho multi-class
              metrics=['accuracy'])

***Thêm Early Stopping***

In [9]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=10,
    restore_best_weights=True
)

***Huấn luyện mô hình***

In [10]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=64,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],
                    verbose=1)

Epoch 1/100
188/188 [==============================] - 2s 4ms/step - loss: 0.1473 - accuracy: 0.9554 - val_loss: 0.0576 - val_accuracy: 0.9787
Epoch 2/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0342 - accuracy: 0.9881 - val_loss: 0.0337 - val_accuracy: 0.9865
Epoch 3/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0219 - accuracy: 0.9927 - val_loss: 0.0224 - val_accuracy: 0.9915
Epoch 4/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0172 - accuracy: 0.9944 - val_loss: 0.0141 - val_accuracy: 0.9960
Epoch 5/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0164 - accuracy: 0.9945 - val_loss: 0.0187 - val_accuracy: 0.9933
Epoch 6/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 0.9958 - val_loss: 0.0125 - val_accuracy: 0.9945
Epoch 7/100
188/188 [==============================] - 0s 2ms/step - loss: 0.0139 - accuracy: 0.9948 - val_loss: 0.0098 - val_accuracy: 0.9983

***Đánh giá mô hình trên tập test***

In [11]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

125/125 [==============================] - 0s 2ms/step - loss: 0.0029 - accuracy: 0.9995
Test Loss: 0.0028953542932868004, Test Accuracy: 0.9994999766349792


***Dự đoán trên tập test***

In [12]:
# Dự đoán trên tập test
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Chuyển từ xác suất sang nhãn lớp

# In ra một số kết quả dự đoán đầu tiên
print("Dự đoán nhãn (10 mẫu đầu tiên):", y_pred_classes[:10])
print("Nhãn thực tế (10 mẫu đầu tiên):", y_test[:10])

# Đánh giá độ chính xác
accuracy = np.mean(y_pred_classes == y_test)
print(f"Độ chính xác trên tập test: {accuracy:.4f}")


125/125 [==============================] - 0s 1ms/step
Dự đoán nhãn (10 mẫu đầu tiên): [1 0 1 0 0 1 1 1 1 1]
Nhãn thực tế (10 mẫu đầu tiên): [1 0 1 0 0 1 1 1 1 1]
Độ chính xác trên tập test: 0.9995


***Lưu mô hình thành file .tflite***

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]  # Giảm kích thước model
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\vuquo\AppData\Local\Temp\tmpxdb0fsug\assets


INFO:tensorflow:Assets written to: C:\Users\vuquo\AppData\Local\Temp\tmpxdb0fsug\assets


3648

In [14]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [15]:
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, c_model_name))

In [17]:
#Thử 1 vài test case
tflite_model_path = "disease_prediction_model.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


labels = [ "Bình thường", 
    "Bất ổn"]

test_samples = np.array([
    [70, 98], [80, 95], [90, 85], [60, 90], [100, 92], [85, 99], [55, 88], [95, 80]
], dtype=np.float32)

print("Testing multiple samples:")
print("-" * 80)
for sample in test_samples:
    sample_scaled = scaler.transform(sample.reshape(1, -1))
    interpreter.set_tensor(input_details[0]['index'], sample_scaled)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output_data[0])
    print(f"Input: {sample}, Scaled: {sample_scaled[0]}")
    print(f"Predicted: {labels[predicted_class]}")
    print(f"Probabilities: {output_data[0]}")
    print("-" * 80)

Testing multiple samples:
--------------------------------------------------------------------------------
Input: [70. 98.], Scaled: [-0.46062526  0.81436235]
Predicted: Bình thường
Probabilities: [9.9998581e-01 1.2891178e-05 5.7135094e-07 6.4931271e-07]
--------------------------------------------------------------------------------
Input: [80. 95.], Scaled: [-0.1431508   0.29468837]
Predicted: Bình thường
Probabilities: [9.6627486e-01 3.3719037e-02 2.6479702e-06 3.4052562e-06]
--------------------------------------------------------------------------------
Input: [90. 85.], Scaled: [ 0.17432363 -1.4375583 ]
Predicted: Bất ổn
Probabilities: [0. 1. 0. 0.]
--------------------------------------------------------------------------------
Input: [60. 90.], Scaled: [-0.7780997 -0.571435 ]
Predicted: Bất ổn
Probabilities: [2.1695616e-29 1.0000000e+00 3.1646829e-32 2.8072636e-32]
--------------------------------------------------------------------------------
Input: [100.  92.], Scaled: [ 0.4